In [1]:
# yelp scraping project
import requests
import json
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
import requests

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import pandas as pd
import re

%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

import warnings
warnings.filterwarnings('ignore')

In [2]:
pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 2500)
pd.set_option('display.width', 1000)

height has been deprecated.



In [3]:
# hard code this section to scrape multiple pages for reviews
main = 'https://www.yelp.com/biz/harbor-barber-huntington-beach-3'
url = [main,
       main + '?start=20',
       main + '?start=40',
       main + '?start=60',
       main + '?start=80',
       main + '?start=100',
       main + '?start=120',
       main + '?start=140',
       main + '?start=160',]
#        main + '?start=180',]
#       main + '?start=200',
#       main + '?start=220',
#       main + '?start=240',
#       main + '?start=260',
#       main + '?start=280',
#       main + '?start=300',]

In [4]:
for i in url:
    print (i)

https://www.yelp.com/biz/harbor-barber-huntington-beach-3
https://www.yelp.com/biz/harbor-barber-huntington-beach-3?start=20
https://www.yelp.com/biz/harbor-barber-huntington-beach-3?start=40
https://www.yelp.com/biz/harbor-barber-huntington-beach-3?start=60
https://www.yelp.com/biz/harbor-barber-huntington-beach-3?start=80
https://www.yelp.com/biz/harbor-barber-huntington-beach-3?start=100
https://www.yelp.com/biz/harbor-barber-huntington-beach-3?start=120
https://www.yelp.com/biz/harbor-barber-huntington-beach-3?start=140
https://www.yelp.com/biz/harbor-barber-huntington-beach-3?start=160


# Start Review Section

In [5]:
reviews = []

In [6]:
def get_review_content(url):
    for i in url:
        response = requests.get(i)
        soup = BeautifulSoup(response.text, 'html.parser')
        
        rc_unstripped = soup.find_all(attrs={'class': 'review-content'})
        rc_length = (len(rc_unstripped))
        
        for i in range (0, rc_length):
            reviews.append(rc_unstripped[i].text.strip())       

In [7]:
# Run the function
get_review_content(url)

In [8]:
print(len(reviews))

178


In [9]:
print(reviews[2])

3/22/2018
    








1 check-in
        


They fixed my son's hair after another barber shop sort of butchered it. They were funny, really nice and gave him a great haircut, even offered him root beer. :) I think this is the new barber shop!








Good reading while we wait for a cut :)


Good reading while we wait for a cut :)


In [10]:
# convert reviews to pd series
reviews_s = pd.Series(reviews)
reviews_s.head()

0    1/3/2018\n    \n\n\n\n\n\n\n\n\n1 check-in\n  ...
1    4/8/2018\n    \n\nAmber and the staff are grea...
2    3/22/2018\n    \n\n\n\n\n\n\n\n\n1 check-in\n ...
3    4/9/2018\n    \n\nMike is the man.  I've been ...
4    3/31/2018\n    \n\nEven my beard was giddy to ...
dtype: object

In [11]:
# create a pandas df from reviews
data = {'Reviews' : reviews_s}

df = pd.DataFrame.from_dict(data)
df = df[['Reviews']]    

In [12]:
df.head()

,Reviews
0,1/3/2018\n \n\n\n\n\n\n\n\n\n1 check-in\n ...
1,4/8/2018\n \n\nAmber and the staff are grea...
2,3/22/2018\n \n\n\n\n\n\n\n\n\n1 check-in\n ...
3,4/9/2018\n \n\nMike is the man. I've been ...
4,3/31/2018\n \n\nEven my beard was giddy to ...


In [13]:
# split date and reviews
df['Date'] = df.Reviews.str.split('\n').str.get(0)
df['Review'] = df.Reviews.str.split('\n\n').str.get(1)

In [14]:
# drop Reviews column
df.drop('Reviews', axis=1, inplace=True)

In [15]:
df.head()

,Date,Review
0,1/3/2018,
1,4/8/2018,Amber and the staff are great. Easy going plac...
2,3/22/2018,
3,4/9/2018,Mike is the man. I've been going to him for y...
4,3/31/2018,Even my beard was giddy to get its first trim!...


In [16]:
# delete NaN rows
df2 = df.replace(r"", np.nan)
df2.tail()

,Date,Review
173,12/17/2014,First visit in and i can't ask for anything mo...
174,10/10/2014,best barber shop around! greg and jason are gr...
175,7/11/2014,"FINALLY, A GENTLEMAN'S BARBER SHOP. No air ho..."
176,7/6/2014,NaN
177,6/22/2014,My husband and son both get their hair cut her...


In [17]:
# drop NaN rows
df3 = df2.dropna()

# reset index
df3.reset_index(drop=True, inplace=True)

df3.head()
df3.shape

,Date,Review
0,4/8/2018,Amber and the staff are great. Easy going plac...
1,4/9/2018,Mike is the man. I've been going to him for y...
2,3/31/2018,Even my beard was giddy to get its first trim!...
3,4/7/2018,This place is amazing! We were greeted immedia...
4,3/2/2018,If you're looking for a quality cut this is th...


(138, 2)

In [18]:
# checks to see if there are any null values
df3.isnull().sum()

Date      0
Review    0
dtype: int64

# Pandas DF and Sentiment Section

In [19]:
from textblob import TextBlob

In [20]:
# convert Review column in df3 to list
review_list = df3['Review'].tolist()
review_list[0]

'Amber and the staff are great. Easy going place. Highly recommended to anyone in the area'

In [21]:
len(review_list)

138

In [22]:
sentiment_score = []
# iterate through review list
for r in review_list:
    sentiment = TextBlob(r)
    sentiment_score.append(sentiment.sentiment.polarity)

In [23]:
# convert to series
sentiment_score_convert = pd.Series(sentiment_score)

In [24]:
# add sentiment score to sentiment_df
df3.insert(loc=1, column='SentimentScore', value=sentiment_score_convert)

In [25]:
df3.head()

,Date,SentimentScore,Review
0,4/8/2018,0.464444,Amber and the staff are great. Easy going plac...
1,4/9/2018,0.396429,Mike is the man. I've been going to him for y...
2,3/31/2018,0.437500,Even my beard was giddy to get its first trim!...
3,4/7/2018,0.472821,This place is amazing! We were greeted immedia...
4,3/2/2018,0.091667,If you're looking for a quality cut this is th...


In [26]:
sentiment = []
# iterate through sentiment_score to assign a pos or neg val
for s in sentiment_score:
    if s > 0:
        sentiment.append('Positive')
    else:
        sentiment.append('Negative')

In [27]:
print(len(sentiment_score))
print(len(sentiment))

138
138


In [28]:
# check for null values
df3.isnull().sum()

Date              0
SentimentScore    0
Review            0
dtype: int64

In [29]:
# convert sentiment to a pandas series
sentiment_2 = pd.Series(sentiment)

In [30]:
# add sentiment score to sentiment_df
df3.insert(loc=2, column='Sentiment', value=sentiment_2)

In [31]:
df3.head()

,Date,SentimentScore,Sentiment,Review
0,4/8/2018,0.464444,Positive,Amber and the staff are great. Easy going plac...
1,4/9/2018,0.396429,Positive,Mike is the man. I've been going to him for y...
2,3/31/2018,0.437500,Positive,Even my beard was giddy to get its first trim!...
3,4/7/2018,0.472821,Positive,This place is amazing! We were greeted immedia...
4,3/2/2018,0.091667,Positive,If you're looking for a quality cut this is th...


In [32]:
# check value counts for Positive and Negative
df3.Sentiment.value_counts()

Positive    135
Negative      3
Name: Sentiment, dtype: int64

In [33]:
# convert Sentiment label to a numerical value
df3['SentimentNum'] = df3.Sentiment.map({'Positive': 0,
                                                 'Negative': 1})

In [34]:
df3.head()
df3.tail()
df3.shape

,Date,SentimentScore,Sentiment,Review,SentimentNum
0,4/8/2018,0.464444,Positive,Amber and the staff are great. Easy going plac...,0
1,4/9/2018,0.396429,Positive,Mike is the man. I've been going to him for y...,0
2,3/31/2018,0.437500,Positive,Even my beard was giddy to get its first trim!...,0
3,4/7/2018,0.472821,Positive,This place is amazing! We were greeted immedia...,0
4,3/2/2018,0.091667,Positive,If you're looking for a quality cut this is th...,0


,Date,SentimentScore,Sentiment,Review,SentimentNum
133,11/15/2014,0.506169,Positive,Moved to HB from Santa Barbara and while I had...,0
134,12/17/2014,0.338889,Positive,First visit in and i can't ask for anything mo...,0
135,10/10/2014,0.648214,Positive,best barber shop around! greg and jason are gr...,0
136,7/11/2014,0.111667,Positive,"FINALLY, A GENTLEMAN'S BARBER SHOP. No air ho...",0
137,6/22/2014,0.650000,Positive,My husband and son both get their hair cut her...,0


(138, 5)

In [35]:
df3.SentimentNum.value_counts()

0    135
1      3
Name: SentimentNum, dtype: int64

In [36]:
# extract business name from "main" url
bizname_split = main.split('https://www.yelp.com/biz/')
bizname = bizname_split[1]
bizname

'harbor-barber-huntington-beach-3'

In [37]:
# save csv to reviews folder to use for NLP
df3.to_csv(f'reviews/{bizname}.csv', index=False)

# Section for NLTK

### Import saved csvs from reviews directory

In [ ]:
# Define X and y for use with COUNTVECTORIZE
X = df3.Review
y = df3.SentimentNum
print(X.shape)
print(y.shape)